In [38]:
raw = pd.read_sql("""
                    select 
                    tanggal_data, substring(nomor_rekening_loan,1,10) as norek, nomor_rekening_loan, cif_no, 
                    tanggal_buka, tanggal_tutup, tipe_segment, saldo_psak, plafond, usia_hari, usia_jatuh_tempo, 
                    tunggak_pokok, tunggak_bunga, denda, frekuensi_tunggakan, angsuran, saldo as saldo_tabungan, 
                    kolek_nasabah 
                    from mis.df_kredit_monthly
                    where kolek_nasabah is not null and
                    status_rekening = '1'
                    """,conn)

C:\Users\User\AppData\Local\Temp\ipykernel_10320\2288075991.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  raw = pd.read_sql("""


In [39]:
makro = pd.read_sql("""
                   select *
                    from mis.df_makro 
                    """,conn)

C:\Users\User\AppData\Local\Temp\ipykernel_10320\745890864.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  makro = pd.read_sql("""


In [40]:
df = pd.merge(raw, makro, left_on='tanggal_data', right_on='tanggal_data', how='left')

df['saldo_tabungan'] = df['saldo_tabungan'].fillna(0)

In [46]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import numpy as np

In [47]:
df = pd.read_csv('sample_data_2024.csv')

In [54]:
df['tanggal_data'].value_counts()

tanggal_data
2024-09-30    97
2024-07-31    97
2024-02-29    91
2024-03-31    88
2024-08-31    84
2024-11-30    84
2024-10-31    83
2024-04-30    80
2024-05-31    79
2024-01-31    77
2024-12-31    74
2024-06-30    66
Name: count, dtype: int64

In [55]:
label_encoder = LabelEncoder()

df['kolek_nasabah_encoded'] = label_encoder.fit_transform(df['kolek_nasabah'])

In [56]:
df.head()

,tanggal_data,norek,nomor_rekening_loan,cif_no,tanggal_buka,tanggal_tutup,tipe_segment,saldo_psak,plafond,usia_hari,...,saldo_tabungan,kolek_nasabah,inflasi,suku_bunga_acuan,bi_usd_jual,bi_usd_beli,id_tpt,id_ikk,id_ihp,kolek_nasabah_encoded
0,2024-09-30,6551218611,65512186112,411533701979,2024-01-10,1900-01-01,Konsumer,5.948410e+08,8.467082e+08,2999,...,4726853.44,5,0.0218,0.05,16011.71,16000.87,0.0435,122.63,119.73,4
1,2024-04-30,7657591293,76575912931,813968990198,2024-10-02,1900-01-01,Konsumer,5.120916e+08,6.231213e+08,792,...,17694275.34,3,0.0223,0.06,16688.15,15973.41,0.0581,120.44,124.24,2
2,2024-08-31,2495486473,24954864731,395255075866,2024-06-17,1900-01-01,Konsumer,3.766334e+08,4.017521e+08,2706,...,5080224.97,3,0.0200,0.04,16456.03,16181.85,0.0527,122.56,120.97,2
3,2024-09-30,7115548600,71155486008,277383226771,2024-03-15,1900-01-01,Konsumer,6.738211e+08,8.753350e+08,3479,...,18769231.41,2,0.0299,0.06,16182.52,16732.24,0.0481,121.59,116.52,1
4,2024-08-31,4078382646,40783826461,288234878607,2024-12-23,1900-01-01,Konsumer,1.334227e+07,1.599025e+07,4752,...,1414590.18,4,0.0221,0.05,16420.46,15949.06,0.0612,121.73,113.94,3


In [57]:
columns_to_drop = ['norek', 'cif_no', 'tanggal_buka', 'tanggal_tutup', 'tipe_segment', 'kolek_nasabah']  # Replace with your column names

# Drop the specified columns
data = df.drop(columns=columns_to_drop)

X = data.drop(['tanggal_data','nomor_rekening_loan','kolek_nasabah_encoded'], axis=1)
y = data['kolek_nasabah_encoded']

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

# Inisialisasi scaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

# Hitung bobot kelas secara otomatis
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = dict(enumerate(class_weights))

# Definisikan model dengan class_weight
model = lgb.LGBMClassifier(class_weight=class_weight_dict)
model.fit(X_train_scaled, y_train)

# Predict on validation set
y_valid_pred = model.predict(X_valid_scaled)

# Predict on test set
y_test_pred = model.predict(X_test_scaled)

y_train_pred = model.predict(X_train_scaled)
y_val_pred = model.predict(X_valid_scaled)
y_test_pred = model.predict(X_test_scaled)

df_train = X_train
df_train['kolek_nasabah'] = y_train
df_train['kolek_prediksi'] = y_train_pred

df_test = X_test
df_test['kolek_nasabah'] = y_test
df_test['kolek_prediksi'] = y_test_pred

df_val = X_valid
df_val['kolek_nasabah'] = y_valid
df_val['kolek_prediksi'] = y_val_pred

C:\Users\Hanif\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\Hanif\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\Hanif\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\Hanif\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\Hanif\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' wa

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000741 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3578
[LightGBM] [Info] Number of data points in the train set: 700, number of used features: 17
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

In [58]:
def label_performance(row):
    if abs(row['kolek_prediksi'] - row['kolek_nasabah']) > 2:
        return 0
    elif abs(row['kolek_prediksi'] - row['kolek_nasabah']) < 2:
        return 1
    else:
        return 0 # or handle the equal case as needed

df_train['model_performance'] = df_train.apply(label_performance, axis=1)
df_val['model_performance'] = df_val.apply(label_performance, axis=1)
df_test['model_performance'] = df_test.apply(label_performance, axis=1)


In [59]:
df_combined = pd.concat([df_train, df_test, df_val], ignore_index=True)

# Ambil 7 kolom pertama dari df
df_subset = df.iloc[:, :7]

# Gabungkan kedua DataFrame berdasarkan index
df_result = pd.concat([df_subset, df_combined], axis=1)

value_mapping = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5}

# Replace the values in both 'kolek_nasabah' and 'kolek_prediksi' columns
df_result[['kolek_nasabah', 'kolek_prediksi']] = df_result[['kolek_nasabah', 'kolek_prediksi']].replace(value_mapping)

def update_kolek_prediksi(row):
    if row['kolek_prediksi'] > row['kolek_nasabah']:
        return row['kolek_nasabah'] + 1
    elif row['kolek_nasabah'] > row['kolek_prediksi']:
        return min(row['kolek_nasabah'], row['kolek_prediksi'])
    else:
        return row['kolek_prediksi']

# Apply the rule to update 'kolek_prediksi'
df_result['kolek_prediksi'] = df_result.apply(update_kolek_prediksi, axis=1)

In [60]:
df_result.head()

,tanggal_data,norek,nomor_rekening_loan,cif_no,tanggal_buka,tanggal_tutup,tipe_segment,saldo_psak,plafond,usia_hari,...,inflasi,suku_bunga_acuan,bi_usd_jual,bi_usd_beli,id_tpt,id_ikk,id_ihp,kolek_nasabah,kolek_prediksi,model_performance
0,2024-09-30,6551218611,65512186112,411533701979,2024-01-10,1900-01-01,Konsumer,4.359511e+08,5.109421e+08,2490,...,0.0295,0.05,16575.71,16886.57,0.0614,123.25,110.91,5,5,1
1,2024-04-30,7657591293,76575912931,813968990198,2024-10-02,1900-01-01,Konsumer,1.858130e+08,1.997415e+08,3972,...,0.0397,0.07,16434.42,15984.64,0.0406,122.37,111.25,2,2,1
2,2024-08-31,2495486473,24954864731,395255075866,2024-06-17,1900-01-01,Konsumer,3.901256e+08,5.055596e+08,1953,...,0.0163,0.05,16876.64,15908.55,0.0541,125.85,112.51,4,4,1
3,2024-09-30,7115548600,71155486008,277383226771,2024-03-15,1900-01-01,Konsumer,1.019989e+07,1.503522e+07,2122,...,0.0259,0.06,16512.26,16531.27,0.0369,128.20,115.77,1,1,1
4,2024-08-31,4078382646,40783826461,288234878607,2024-12-23,1900-01-01,Konsumer,3.037269e+08,4.170232e+08,4216,...,0.0340,0.05,16726.77,15904.40,0.0337,122.60,112.84,4,4,1


In [61]:
count_per_tanggal = df_result.groupby('tanggal_data')['nomor_rekening_loan'].size().reset_index(name='count_norek')

# Sort the result by 'tanggal_data' in ascending order
count_per_tanggal_sorted = count_per_tanggal.sort_values(by='tanggal_data', ascending=True)

count_per_tanggal_sorted


,tanggal_data,count_norek
0,2024-01-31,77
1,2024-02-29,91
2,2024-03-31,88
3,2024-04-30,80
4,2024-05-31,79
5,2024-06-30,66
6,2024-07-31,97
7,2024-08-31,84
8,2024-09-30,97
9,2024-10-31,83


In [92]:
df = pd.read_csv('sample_data_2024.csv')

In [122]:
import pandas as pd
import lightgbm as lgb

# Drop unnecessary columns
# Transform 'tanggal_data' to numerical features
forecast = df
forecast['tanggal_data'] = pd.to_datetime(forecast['tanggal_data'])
forecast['year'] = forecast['tanggal_data'].dt.year
forecast['month'] = forecast['tanggal_data'].dt.month
forecast['day'] = forecast['tanggal_data'].dt.day

# Create lag features
lag_features = ['usia_hari', 'tunggak_pokok', 'tunggak_bunga', 'frekuensi_tunggakan', 'suku_bunga_acuan']

for feature in lag_features:
    for lag in range(1, 4):  # Lag hingga 3 bulan
        forecast[f'{feature}_lag{lag}'] = forecast.groupby('nomor_rekening_loan')[feature].shift(lag)

columns_to_drop = ['norek', 'tanggal_buka', 'tanggal_tutup', 'tipe_segment','kolek_nasabah_encoded','tanggal_data','nomor_rekening_loan']
forecast = forecast.drop(columns=columns_to_drop)
forecast = forecast.fillna(0)

# Function to split data
def split_data(forecast):
    # Define the last month and threshold date
    last_year_month = forecast[['year', 'month']].max()
    last_month_start = pd.Timestamp(year=last_year_month['year'], month=last_year_month['month'], day=1)

    # Split the data
    train_data = forecast[(forecast['year'] < last_month_start.year) | 
                          ((forecast['year'] == last_month_start.year) & (forecast['month'] < last_month_start.month))]
    test_data = forecast[(forecast['year'] == last_month_start.year) & (forecast['month'] == last_month_start.month)]

    return train_data, test_data

# Split the data
train_data, test_data = split_data(forecast)

# Define features and target
target = 'kolek_nasabah'

X_train = train_data.drop(columns=target)
y_train = train_data[target]
X_test = test_data.drop(columns=target)
y_test = test_data[target]

# Train the LightGBM model
model = lgb.LGBMClassifier(random_state=42, class_weight='balanced')
model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000487 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4177
[LightGBM] [Info] Number of data points in the train set: 9136, number of used features: 20
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438


LGBMClassifier(class_weight='balanced', random_state=42)

In [123]:
X_train

,cif_no,saldo_psak,plafond,usia_hari,usia_jatuh_tempo,tunggak_pokok,tunggak_bunga,denda,frekuensi_tunggakan,angsuran,...,tunggak_pokok_lag3,tunggak_bunga_lag1,tunggak_bunga_lag2,tunggak_bunga_lag3,frekuensi_tunggakan_lag1,frekuensi_tunggakan_lag2,frekuensi_tunggakan_lag3,suku_bunga_acuan_lag1,suku_bunga_acuan_lag2,suku_bunga_acuan_lag3
0,735568142778,3.785762e+08,4.856146e+08,4148,943.530759,869257.82,399010.78,93.59,7.0,5182263.34,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,264953552937,3.684778e+08,4.319871e+08,2487,1742.523896,423215.71,490529.43,4420.10,6.3,3706619.18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,145283414714,5.150748e+08,7.517128e+08,3520,1794.566815,424474.59,168441.93,2438.61,7.0,6001494.29,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,249986784180,5.504830e+08,7.421217e+08,4559,205.111365,655647.63,762752.91,161.57,0.6,2715738.16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,732772112124,6.826619e+08,9.503188e+08,2770,129.640491,833562.46,345686.85,3697.05,3.5,3300354.17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,759520880772,1.803124e+08,2.127036e+08,4815,1645.643857,992099.36,657471.46,3157.57,8.4,2883814.74,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9995,834091462896,2.374680e+08,2.641714e+08,795,1727.606598,932856.16,376627.41,874.15,5.1,9510212.50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9997,212139341293,1.467086e+08,1.759000e+08,53,549.938462,362910.53,104403.84,2912.96,8.5,4796510.24,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9998,274653914129,5.833298e+07,8.276463e+07,1818,154.565016,491472.92,604637.48,2720.70,5.1,5321226.89,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [124]:
#export model
import pickle

# Save the model to a pickle file
with open('lgb_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [125]:
df = pd.read_csv('sample_data_test_2024.csv')
# Transform 'tanggal_data' to numerical features
df['tanggal_data'] = pd.to_datetime( df['tanggal_data'])
df['year'] =  df['tanggal_data'].dt.year
df['month'] =  df['tanggal_data'].dt.month
df['day'] =  df['tanggal_data'].dt.day

# Sort and reset index
df =  df.sort_values(by=['nomor_rekening_loan', 'year', 'month', 'day']).reset_index(drop=True)

# Create lag features
lag_features = ['usia_hari', 'tunggak_pokok', 'tunggak_bunga', 'frekuensi_tunggakan', 'suku_bunga_acuan']

for feature in lag_features:
    for lag in range(1, 4):  # Lag hingga 3 bulan
         df[f'{feature}_lag{lag}'] =  df.groupby('nomor_rekening_loan')[feature].shift(lag)

df =  df.fillna(0)

In [139]:
from datetime import datetime, timedelta
def start_of_next_month(df):
    # Get the current date
    today = max(df['tanggal_data'])

    # Calculate the first day of the next month
    if today.month == 12:
        next_month = datetime(today.year + 1, 1, 1)
    else:
        next_month = datetime(today.year, today.month + 1, 1)

    return next_month

In [140]:
pred_date = start_of_next_month(df)

In [141]:
pred_date

datetime.datetime(2025, 1, 1, 0, 0)

In [126]:
data_pred = df[df['tanggal_data'] == max(df['tanggal_data'])]
columns_to_drop = ['norek', 'tanggal_buka', 'tanggal_tutup', 'tipe_segment','kolek_nasabah_encoded','tanggal_data','nomor_rekening_loan','kolek_nasabah']
data_pred.drop(columns=columns_to_drop, inplace=True)
data_pred

C:\Users\Hanif\AppData\Local\Temp\ipykernel_19916\2196564614.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_pred.drop(columns=columns_to_drop, inplace=True)


,cif_no,saldo_psak,plafond,usia_hari,usia_jatuh_tempo,tunggak_pokok,tunggak_bunga,denda,frekuensi_tunggakan,angsuran,...,tunggak_pokok_lag3,tunggak_bunga_lag1,tunggak_bunga_lag2,tunggak_bunga_lag3,frekuensi_tunggakan_lag1,frekuensi_tunggakan_lag2,frekuensi_tunggakan_lag3,suku_bunga_acuan_lag1,suku_bunga_acuan_lag2,suku_bunga_acuan_lag3
1,398191548586,3.772697e+08,5.599825e+08,2091,18.330570,21222.34,909751.44,3248.92,9.3,9481919.81,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42,453134730598,4.652399e+08,4.958910e+08,1058,1061.874049,768351.49,60692.41,3474.87,8.5,5715644.04,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43,884115180075,2.932477e+08,3.482659e+08,4768,309.520343,43926.04,475902.36,4528.65,5.1,5609314.29,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46,727766268853,3.283711e+08,3.513974e+08,2562,1504.936518,591321.61,117242.23,3507.17,8.3,3091591.90,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70,862813327498,2.154780e+08,3.125765e+08,4318,887.358941,254289.75,301021.32,1043.00,4.1,4493126.05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9955,710648051999,2.515154e+08,2.905789e+08,43,233.838553,538605.21,970207.91,4022.66,1.8,6738916.66,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9959,329572670426,3.838478e+08,4.031930e+08,4920,211.795566,952150.15,874707.36,3410.50,8.3,1042967.36,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9969,770903032455,7.238960e+07,7.436419e+07,2575,926.343471,529934.24,972892.21,423.99,4.0,7310669.86,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9986,278235489943,1.386725e+08,1.510888e+08,1989,1031.835915,732136.70,414717.90,302.37,3.1,3689474.58,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [127]:
# Load the model from the pickle file
with open('lgb_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# Use the loaded model for prediction
y_pred_loaded = loaded_model.predict(data_pred)

In [146]:
data_pred['tanggal_data'] = pd.to_datetime(data_pred[['year', 'month', 'day']])

# Calculate 'tanggal_prediksi' as the first day of the next month
data_pred['tanggal_prediksi'] = data_pred['tanggal_data'].apply(
    lambda x: datetime(x.year + 1, 1, 1) if x.month == 12 else datetime(x.year, x.month + 1, 1)
)

# Add predictions to the DataFrame
data_pred['kolek_nasabah'] = y_pred_loaded

# Save the DataFrame to CSV
data_pred.to_csv('data_pred.csv', index=False)

C:\Users\Hanif\AppData\Local\Temp\ipykernel_19916\146582833.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_pred['tanggal_data'] = pd.to_datetime(data_pred[['year', 'month', 'day']])
C:\Users\Hanif\AppData\Local\Temp\ipykernel_19916\146582833.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_pred['tanggal_prediksi'] = data_pred['tanggal_data'].apply(
C:\Users\Hanif\AppData\Local\Temp\ipykernel_19916\146582833.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [147]:
data_pred

,cif_no,saldo_psak,plafond,usia_hari,usia_jatuh_tempo,tunggak_pokok,tunggak_bunga,denda,frekuensi_tunggakan,angsuran,...,tunggak_bunga_lag3,frekuensi_tunggakan_lag1,frekuensi_tunggakan_lag2,frekuensi_tunggakan_lag3,suku_bunga_acuan_lag1,suku_bunga_acuan_lag2,suku_bunga_acuan_lag3,kolek_nasabah,tanggal_prediksi,tanggal_data
1,398191548586,3.772697e+08,5.599825e+08,2091,18.330570,21222.34,909751.44,3248.92,9.3,9481919.81,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2025-01-01,2024-12-31
42,453134730598,4.652399e+08,4.958910e+08,1058,1061.874049,768351.49,60692.41,3474.87,8.5,5715644.04,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,2025-01-01,2024-12-31
43,884115180075,2.932477e+08,3.482659e+08,4768,309.520343,43926.04,475902.36,4528.65,5.1,5609314.29,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,2025-01-01,2024-12-31
46,727766268853,3.283711e+08,3.513974e+08,2562,1504.936518,591321.61,117242.23,3507.17,8.3,3091591.90,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,2025-01-01,2024-12-31
70,862813327498,2.154780e+08,3.125765e+08,4318,887.358941,254289.75,301021.32,1043.00,4.1,4493126.05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2025-01-01,2024-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9955,710648051999,2.515154e+08,2.905789e+08,43,233.838553,538605.21,970207.91,4022.66,1.8,6738916.66,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,2025-01-01,2024-12-31
9959,329572670426,3.838478e+08,4.031930e+08,4920,211.795566,952150.15,874707.36,3410.50,8.3,1042967.36,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,2025-01-01,2024-12-31
9969,770903032455,7.238960e+07,7.436419e+07,2575,926.343471,529934.24,972892.21,423.99,4.0,7310669.86,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,2025-01-01,2024-12-31
9986,278235489943,1.386725e+08,1.510888e+08,1989,1031.835915,732136.70,414717.90,302.37,3.1,3689474.58,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,2025-01-01,2024-12-31
